# Final Project Template

For the final project for this module, you are asked to use ETL together with the skills you have learned about Python and MySQL in the previous modules to understand spending patterns.

This module's project is divided into two main parts: Extract-Transform-Load (ETL) and Analysis and Visualization.

Your challenge in this project is to implement the steps suggested by Dr. Sanchez in his videos throughout the module and prove that you have a a clear understanding of each of them by being able to describe and justify them. You will also be tested on your ability to conduct your own analysis to understand spending patterns.

Before you fill out the project outline template below, make sure you:

- Read through the template completely to understand the instructions for the structure of the project.
- Have a clear understanding of what to do to create a model that will return the results you want to find.
- Use Markdown to edit the template.
- Include any screenshots of your code (both Python and MySQL) and of your program windows (Excel, Terminal, VS Code, MySQL Workbench) to demonstrate your steps.

<div class="alert alert-block alert-success">
The purpose of this Jupyter Notebook is to give you a structure to follow when you are solving your problem and developing your model with Python. Make sure you follow it carefully. You can add more subsections if needed, but remember to fill out every section provided in the template.
</div>

<div class="alert alert-block alert-danger">
Delete all cells above, including this one, before submitting your final Notebook.
</div>

# Title

**Your_Name**

Add the title of your project and delete the default one.

# Index

- [Abstract](#Abstract)
- [1. Introduction](#1.-Introduction)
- [2. Extract-Transform-Load](#2.-Extract-Transform-Load)
    - [2.1 The ETL Process](#2.1-The-ETL-Process)
    - [2.2 Data Exploration](#2.2-Data-Exploration)
    - [2.3 Data Preparation](#2.3-Data-Preparation)
    - [2.4 Read the Data Using Python](#2.4-Reading-the-Data-Using-Python)
         - [2.4.1 Reading Sample Data](#2.4.1-Reading-Sample-Data)
         - [2.4.2 Reading the MRST Data](#2.4.2-Reading-the-MRST-Data)
    - [2.5 Writing an Installation Script](#2.5-Writing-an-Installation-Script)
- [3. Analysis and Visualization](#3.-Project-Description)
    - [3.1 Running Queries in MySQL Workbech](#3.1-Running-Queries-in-MySQL-Workbech)
    - [3.2 Running Queries From Python](#3.2-Running-Queries-From-Python)
    - [3.3 Explore Trends](#3.3-Explore-Trends)
    - [3.4 Explore Percentage Change](#3.4-Explore-Percentage-Change)
    - [3.5 Explore Rolling Time Windows](#3.5-Explore-Rolling-Time-Windows)
- [Conclusion](#Conclusion)
- [References](#References)

[Back to top](#Index)


##  Abstract

This is a brief description (150 words or less) of your analysis and the results of your model. Complete this portion of the template after you are done working on your project.

[Back to top](#Index)


## 1. Introduction

Introduce your project using 300 words or less. Describe all the processes you followed to create your ETL, Analysis, and Visualization project. Start by summarizing the steps that you intend to perform and then elaborate on this section after you have completed your project.

[Back to top](#Index)

## 2. Extract-Transform-Load

For each of the sections below, include a description of the steps you followed. Whenever possible, include screenshots of your code or program windows to demonstrate your steps.

In [57]:
# At the top of your notebook
import pandas as pd
import numpy as np
import os

# Define paths once at the top
BASE_DIR = r"C:\Users\mohya\OneDrive\Documents\Github Projects\MIT PC Data Engineering\Module 8 - ETL"
EXCEL_FILE = os.path.join(BASE_DIR, "mrtssales92-present.xlsx")
RAW_CSV = os.path.join(BASE_DIR, "mrts_2017_data.csv")
PREPARED_CSV = os.path.join(BASE_DIR, "mrts_2017_prepared.csv")
SAMPLE_CSV = os.path.join(BASE_DIR, "sample.csv")

[Back to top](#Index)

### 2.1 The ETL Process

For this project, I implemented the ETL process on the MRTS dataset through the following steps:

1. Extract:
   - Read the Excel file 'mrtssales92-present.xlsx' from local directory
   - Split the reading process into two parts to handle different data structures:
     * First part: Columns A:B (NAICS Code and Business descriptions)
     * Second part: Columns C:O (Monthly sales data)

2. Transform:
   - Removed initial rows to clean up header information
   - Removed unnecessary first row from each dataframe
   - Combined the two dataframes using column-wise concatenation
   - Exported to CSV format for further processing

3. Load:
   - Created a CSV file that will be used to load into MySQL database

In [58]:
# Section 2.1: Extract
def extract_mrts_data():
    try:
        if os.path.exists(EXCEL_FILE):
            print("Found the Excel file...")
            
            df = pd.read_excel(EXCEL_FILE, 
                             sheet_name='2017',
                             usecols='A:B',
                             nrows=111,
                             skiprows=3)
            
            df_1 = pd.read_excel(EXCEL_FILE, 
                               sheet_name='2017',
                               usecols='C:O',
                               nrows=111,
                               skiprows=4)
            
            df = df.iloc[2:].reset_index(drop=True)
            df_1 = df_1.iloc[1:].reset_index(drop=True)
            df_2 = pd.concat([df, df_1], axis=1)
            df_2 = df_2.iloc[:-6]
            
            df_2.to_csv(RAW_CSV, index=False)
            print(f"Data extracted and saved to {RAW_CSV}")
            return df_2
    except Exception as e:
        print(f"Extract error: {str(e)}")
        return None

# Run extraction
raw_df = extract_mrts_data()
print("\nFirst few rows of extracted data:")
raw_df.head(5)

Found the Excel file...
Data extracted and saved to C:\Users\mohya\OneDrive\Documents\Github Projects\MIT PC Data Engineering\Module 8 - ETL\mrts_2017_data.csv

First few rows of extracted data:


,NAICS Code,Kind of Business,Jan. 2017,Feb. 2017,Mar. 2017,Apr. 2017,May 2017,Jun. 2017,Jul. 2017,Aug. 2017,Sep. 2017,Oct. 2017,Nov. 2017,Dec. 2017,TOTAL
0,NaN,"Retail and food services sales, total",421943.0,418315.0,483082,465788,494621,481729,475374,490324,469710,476014,497748,558215,5732863.0
1,NaN,Retail sales and food services excl motor vehi...,336756.0,328901.0,376796,368476,388989,379136,373146,385526,369739,378618,402994,458090,4547167.0
2,NaN,Retail sales and food services excl gasoline s...,386019.0,384105.0,443979,426328,453329,441236,435163,448690,428033,434050,457507,518272,5256711.0
3,NaN,Retail sales and food services excl motor vehi...,300832.0,294691.0,337693,329016,347697,338643,332935,343892,328062,336654,362753,418147,4071015.0
4,NaN,"Retail sales, total",368960.0,365480.0,422891,407165,434450,423034,416900,431822,412578,417830,442077,497027,5040214.0


[Back to top](#Index)

### 2.2 Data Exploration

The data exploration phase revealed several important characteristics of our MRTS dataset:

1. Structure:
   - The dataset contains monthly retail sales data for 2017
   - Each row represents a different retail category or subcategory
   - Monthly sales figures from January to December, plus annual total
   - NAICS codes are used to identify business categories

2. Content Analysis:
   - Primary retail categories include:
     * Motor vehicle dealers
     * Food and beverage stores
     * Clothing stores
     * Electronic shopping
     * Food services
   - Both adjusted and unadjusted sales figures are present
   - Some data points are marked as '(S)' indicating suppressed values

3. Data Quality Observations:
   - Some missing values present in specific categories
   - Mix of numeric and text data
   - Special characters and annotations present
   - Hierarchical structure in business categories

4. Key Metrics:
   - Monthly sales patterns
   - Category-wise distribution
   - Seasonal variations
   - Year-total calculations



In [59]:
# Section 2.2: Explore
def explore_mrts_data():
    df = pd.read_csv(RAW_CSV)
    
    print("\n=== Basic Dataset Information ===")
    print(f"Number of rows: {df.shape[0]}")
    print(f"Number of columns: {df.shape[1]}")
    
    print("\n=== Column Names ===")
    print(df.columns.tolist())
    
    print("\n=== Missing Values ===")
    print(df.isnull().sum())
    
    print("\n=== Checking for Special Values ===")
    special_chars = df.apply(lambda x: x.astype(str).str.contains('\(S\)').any())
    print("Columns with (S) values:")
    print(special_chars[special_chars])
    
    return df

# Run exploration
explored_df = explore_mrts_data()
explored_df


=== Basic Dataset Information ===
Number of rows: 104
Number of columns: 15

=== Column Names ===
['NAICS  Code', 'Kind of Business', 'Jan. 2017', 'Feb. 2017', 'Mar. 2017', 'Apr. 2017', 'May 2017', 'Jun. 2017', 'Jul. 2017', 'Aug. 2017', 'Sep. 2017', 'Oct. 2017', 'Nov. 2017', 'Dec. 2017', 'TOTAL']

=== Missing Values ===
NAICS  Code         15
Kind of Business     0
Jan. 2017            1
Feb. 2017            1
Mar. 2017            1
Apr. 2017            1
May 2017             1
Jun. 2017            1
Jul. 2017            1
Aug. 2017            1
Sep. 2017            1
Oct. 2017            1
Nov. 2017            1
Dec. 2017            1
TOTAL               40
dtype: int64

=== Checking for Special Values ===
Columns with (S) values:
Mar. 2017    True
Apr. 2017    True
May 2017     True
Jun. 2017    True
Jul. 2017    True
Aug. 2017    True
Sep. 2017    True
Oct. 2017    True
Nov. 2017    True
Dec. 2017    True
dtype: bool


,NAICS Code,Kind of Business,Jan. 2017,Feb. 2017,Mar. 2017,Apr. 2017,May 2017,Jun. 2017,Jul. 2017,Aug. 2017,Sep. 2017,Oct. 2017,Nov. 2017,Dec. 2017,TOTAL
0,NaN,"Retail and food services sales, total",421943.0,418315.0,483082,465788,494621,481729,475374,490324,469710,476014,497748,558215,5732863.0
1,NaN,Retail sales and food services excl motor vehi...,336756.0,328901.0,376796,368476,388989,379136,373146,385526,369739,378618,402994,458090,4547167.0
2,NaN,Retail sales and food services excl gasoline s...,386019.0,384105.0,443979,426328,453329,441236,435163,448690,428033,434050,457507,518272,5256711.0
3,NaN,Retail sales and food services excl motor vehi...,300832.0,294691.0,337693,329016,347697,338643,332935,343892,328062,336654,362753,418147,4071015.0
4,NaN,"Retail sales, total",368960.0,365480.0,422891,407165,434450,423034,416900,431822,412578,417830,442077,497027,5040214.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,453,Miscellaneous stores retailers,10110.0,10280.0,10189,10249,10043,10048,10258,10235,10354,10392,10590,10656,NaN
100,454,Nonstore retailers,48132.0,48312.0,48903,49624,49991,50267,50713,51035,51561,51074,53421,54363,NaN
101,4541,Electronic shopping and mail order houses,41456.0,41662.0,42157,42922,43098,43438,43918,44008,44424,44077,46017,46733,NaN
102,45431,Fuel dealers,2425.0,2414.0,2516,2523,2615,2530,2486,2579,2645,2559,2896,3123,NaN


[Back to top](#Index)

### 2.3 Data Preparation

The data preparation phase involved several key transformations:

1. Data Cleaning:
   - Standardized column names by removing extra spaces
   - Replaced suppressed values '(S)' with NaN
   - Converted string numeric values to proper numeric type
   - Handled missing NAICS codes

2. Data Enhancement:
   - Created average monthly sales calculations
   - Identified maximum and minimum sales months for each category
   - Removed rows with all missing monthly values
   - Standardized numeric formats across all sales columns

3. Quality Improvements:
   - Better handling of special characters and annotations
   - Consistent data types across similar columns
   - Proper handling of missing values
   - Enhanced data structure for analysis

4. Output:
   - Created a clean, analysis-ready dataset
   - Added derived features for deeper analysis
   - Maintained data integrity while improving usability
   - Saved prepared data in a new CSV file


In [60]:
# Section 2.3: Prepare
def prepare_mrts_data():
  df = pd.read_csv(RAW_CSV)
  
  print("Starting data preparation...")
  print(f"Initial shape: {df.shape}")
  
  # Clean column names
  df.columns = df.columns.str.strip()
  
  # Replace '(S)' with NaN
  df = df.replace('(S)', np.nan)
  
  # Create two dataframes using index 65
  not_adjusted_df = df.iloc[:65].copy()
  adjusted_df = df.iloc[65:].copy()
  
  # Add Adjustments column to each
  not_adjusted_df['Adjustments'] = 'Not Adjusted'
  adjusted_df['Adjustments'] = 'Adjusted'
  
  # Concatenate back together
  df = pd.concat([not_adjusted_df, adjusted_df], axis=0)
  
  # Reorder columns to put Adjustments as column 2
  columns = df.columns.tolist()
  columns = [columns[0]] + ['Adjustments'] + [col for col in columns if col != 'Adjustments' and col != columns[0]]
  df = df[columns]
  
  # Convert numeric columns
  numeric_columns = df.columns.tolist()[3:]
  
  for col in numeric_columns:
      df[col] = pd.to_numeric(df[col], errors='coerce')
  
  df.to_csv(PREPARED_CSV, index=False)
  print(f"\nPrepared data saved to {PREPARED_CSV}")
  
  print("\nSample of prepared data:")
  print(df.head())
  
  return df

# Run preparation
prepared_df = prepare_mrts_data()
prepared_df

Starting data preparation...
Initial shape: (104, 15)

Prepared data saved to C:\Users\mohya\OneDrive\Documents\Github Projects\MIT PC Data Engineering\Module 8 - ETL\mrts_2017_prepared.csv

Sample of prepared data:
  NAICS  Code   Adjustments  \
0         NaN  Not Adjusted   
1         NaN  Not Adjusted   
2         NaN  Not Adjusted   
3         NaN  Not Adjusted   
4         NaN  Not Adjusted   

                                    Kind of Business  Jan. 2017  Feb. 2017  \
0              Retail and food services sales, total   421943.0   418315.0   
1  Retail sales and food services excl motor vehi...   336756.0   328901.0   
2  Retail sales and food services excl gasoline s...   386019.0   384105.0   
3  Retail sales and food services excl motor vehi...   300832.0   294691.0   
4                                Retail sales, total   368960.0   365480.0   

   Mar. 2017  Apr. 2017  May 2017  Jun. 2017  Jul. 2017  Aug. 2017  Sep. 2017  \
0   483082.0   465788.0  494621.0   481729.0   

,NAICS Code,Adjustments,Kind of Business,Jan. 2017,Feb. 2017,Mar. 2017,Apr. 2017,May 2017,Jun. 2017,Jul. 2017,Aug. 2017,Sep. 2017,Oct. 2017,Nov. 2017,Dec. 2017,TOTAL
0,NaN,Not Adjusted,"Retail and food services sales, total",421943.0,418315.0,483082.0,465788.0,494621.0,481729.0,475374.0,490324.0,469710.0,476014.0,497748.0,558215.0,5732863.0
1,NaN,Not Adjusted,Retail sales and food services excl motor vehi...,336756.0,328901.0,376796.0,368476.0,388989.0,379136.0,373146.0,385526.0,369739.0,378618.0,402994.0,458090.0,4547167.0
2,NaN,Not Adjusted,Retail sales and food services excl gasoline s...,386019.0,384105.0,443979.0,426328.0,453329.0,441236.0,435163.0,448690.0,428033.0,434050.0,457507.0,518272.0,5256711.0
3,NaN,Not Adjusted,Retail sales and food services excl motor vehi...,300832.0,294691.0,337693.0,329016.0,347697.0,338643.0,332935.0,343892.0,328062.0,336654.0,362753.0,418147.0,4071015.0
4,NaN,Not Adjusted,"Retail sales, total",368960.0,365480.0,422891.0,407165.0,434450.0,423034.0,416900.0,431822.0,412578.0,417830.0,442077.0,497027.0,5040214.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,453,Adjusted,Miscellaneous stores retailers,10110.0,10280.0,10189.0,10249.0,10043.0,10048.0,10258.0,10235.0,10354.0,10392.0,10590.0,10656.0,NaN
100,454,Adjusted,Nonstore retailers,48132.0,48312.0,48903.0,49624.0,49991.0,50267.0,50713.0,51035.0,51561.0,51074.0,53421.0,54363.0,NaN
101,4541,Adjusted,Electronic shopping and mail order houses,41456.0,41662.0,42157.0,42922.0,43098.0,43438.0,43918.0,44008.0,44424.0,44077.0,46017.0,46733.0,NaN
102,45431,Adjusted,Fuel dealers,2425.0,2414.0,2516.0,2523.0,2615.0,2530.0,2486.0,2579.0,2645.0,2559.0,2896.0,3123.0,NaN


[Back to top](#Index)

### 2.4 Read the Data Using Python

Python offers a powerful library called pandas that makes reading CSV (Comma-Separated Values) files straightforward and efficient. The primary method is `pd.read_csv()`, which can handle various file formats and configurations.

Key features of pandas' CSV reading capabilities:

1. Basic Reading:
```python
# Simple read of CSV file
filepath = r"pathfile"

def read_csv_file(filepath: str, usecols: str = None, skiprows: int = None, nrows: int = None):                
   try:
       df = pd.read_csv(filepath, 
                       usecols=usecols,
                       skiprows=skiprows,
                       nrows=nrows)
       print(f"Successfully read CSV file from: {filepath}")
       print(f"DataFrame shape: {df.shape}")
       return df
       
   except FileNotFoundError:
       print(f"Error: File not found at {filepath}")
       return None
   except Exception as e:
       print(f"Error reading CSV file: {str(e)}")
       return None
```



[Back to top](#Index)

### 2.4.1 Reading Sample Data

First create the sample.csv file
```python
# First, read the prepared CSV
df_prepared = pd.read_csv(PREPARED_CSV)

# Get first 5 rows
sample_df = df_prepared.head(5)

# Create sample file path
SAMPLE_CSV = os.path.join(BASE_DIR, "sample.csv")

# Save to new CSV file
sample_df.to_csv(SAMPLE_CSV, index=False)
print(f"Sample CSV created at: {SAMPLE_CSV}")
print("\nSample data:")
print(sample_df)
```

To read the sample dataset using Python, I took the following approach:

1. First, I defined a reusable function called `read_csv_file` that accepts several parameters:
  - `filepath`: The path to the CSV file (required)
  - `usecols`: Which columns to read (optional)
  - `skiprows`: How many rows to skip (optional)
  - `nrows`: How many rows to read (optional)

2. Then, I used this function to read the sample data with a simple call:
```python
sample_df = read_csv_file(SAMPLE_CSV)

In [61]:
def read_csv_file(filepath: str, usecols: str = None, skiprows: int = None, nrows: int = None):                
   try:
       df = pd.read_csv(filepath, 
                       usecols=usecols,
                       skiprows=skiprows,
                       nrows=nrows)
       print(f"Successfully read CSV file from: {filepath}")
       print(f"DataFrame shape: {df.shape}")
       return df
       
   except FileNotFoundError:
       print(f"Error: File not found at {filepath}")
       return None
   except Exception as e:
       print(f"Error reading CSV file: {str(e)}")
       return None
   
sample_df = read_csv_file(SAMPLE_CSV)
sample_df

Successfully read CSV file from: C:\Users\mohya\OneDrive\Documents\Github Projects\MIT PC Data Engineering\Module 8 - ETL\sample.csv
DataFrame shape: (5, 15)


,NAICS Code,Kind of Business,Jan. 2017,Feb. 2017,Mar. 2017,Apr. 2017,May 2017,Jun. 2017,Jul. 2017,Aug. 2017,Sep. 2017,Oct. 2017,Nov. 2017,Dec. 2017,TOTAL
0,NaN,NOT ADJUSTED,421943.0,418315.0,483082.0,465788.0,494621.0,481729.0,475374.0,490324.0,469710.0,476014.0,497748.0,558215.0,5732863.0
1,NaN,"Retail and food services sales, total",336756.0,328901.0,376796.0,368476.0,388989.0,379136.0,373146.0,385526.0,369739.0,378618.0,402994.0,458090.0,4547167.0
2,NaN,Retail sales and food services excl motor vehi...,386019.0,384105.0,443979.0,426328.0,453329.0,441236.0,435163.0,448690.0,428033.0,434050.0,457507.0,518272.0,5256711.0
3,NaN,Retail sales and food services excl gasoline s...,300832.0,294691.0,337693.0,329016.0,347697.0,338643.0,332935.0,343892.0,328062.0,336654.0,362753.0,418147.0,4071015.0
4,NaN,Retail sales and food services excl motor vehi...,368960.0,365480.0,422891.0,407165.0,434450.0,423034.0,416900.0,431822.0,412578.0,417830.0,442077.0,497027.0,5040214.0


[Back to top](#Index)

### 2.4.2 Reading the MRTS Data

Using the same reading function we created earlier, we now read the full MRTS dataset. This demonstrates the flexibility of our function - it works the same way whether reading a small sample or the complete dataset. The function:

Takes our prepared CSV file path as input
Reads all columns since we didn't specify usecols
Reads all rows since we didn't specify nrows or skiprows
Returns the complete dataset as a pandas DataFrame

In [62]:
mrts_df = read_csv_file(PREPARED_CSV)
mrts_df.head()

Successfully read CSV file from: C:\Users\mohya\OneDrive\Documents\Github Projects\MIT PC Data Engineering\Module 8 - ETL\mrts_2017_prepared.csv
DataFrame shape: (104, 16)


,NAICS Code,Adjustments,Kind of Business,Jan. 2017,Feb. 2017,Mar. 2017,Apr. 2017,May 2017,Jun. 2017,Jul. 2017,Aug. 2017,Sep. 2017,Oct. 2017,Nov. 2017,Dec. 2017,TOTAL
0,NaN,Not Adjusted,"Retail and food services sales, total",421943.0,418315.0,483082.0,465788.0,494621.0,481729.0,475374.0,490324.0,469710.0,476014.0,497748.0,558215.0,5732863.0
1,NaN,Not Adjusted,Retail sales and food services excl motor vehi...,336756.0,328901.0,376796.0,368476.0,388989.0,379136.0,373146.0,385526.0,369739.0,378618.0,402994.0,458090.0,4547167.0
2,NaN,Not Adjusted,Retail sales and food services excl gasoline s...,386019.0,384105.0,443979.0,426328.0,453329.0,441236.0,435163.0,448690.0,428033.0,434050.0,457507.0,518272.0,5256711.0
3,NaN,Not Adjusted,Retail sales and food services excl motor vehi...,300832.0,294691.0,337693.0,329016.0,347697.0,338643.0,332935.0,343892.0,328062.0,336654.0,362753.0,418147.0,4071015.0
4,NaN,Not Adjusted,"Retail sales, total",368960.0,365480.0,422891.0,407165.0,434450.0,423034.0,416900.0,431822.0,412578.0,417830.0,442077.0,497027.0,5040214.0


[Back to top](#Index)

### 2.5 Writing an Installation Script

The script handles:

    -Secure database connection using configuration file
    -Appropriate data type mapping for each column
    -Proper handling of missing values (NaN to NULL)
    -Data cleaning before insertion
    -Error handling at each step

In [63]:
# Section 2.5: Writing an Installation Script

import mysql.connector
import yaml
import pandas as pd
import numpy as np

def create_database():
   try:
       # Load database configuration
       db = yaml.safe_load(open('db.yaml'))
       config = {
           'user': db['user'],
           'password': db['password'],
           'host': db['host'],
           'auth_plugin': 'mysql_native_password'
       }
       
       # Create connection
       cnx = mysql.connector.connect(**config)
       cursor = cnx.cursor()
       
       # Create database if it doesn't exist
       cursor.execute("CREATE DATABASE IF NOT EXISTS retail_sales")
       print("Database 'retail_sales' created or already exists")
       
       # Close initial connection
       cursor.close()
       cnx.close()
       
       # Reconnect with the new database
       config['database'] = 'retail_sales'
       cnx = mysql.connector.connect(**config)
       cursor = cnx.cursor()
       
       print("Connected to retail_sales database")
       return cnx, cursor
       
   except Exception as e:
       print(f"Error: {str(e)}")
       return None, None

def prepare_data_for_mysql(df):
   """Clean and prepare data for MySQL loading"""
   # Make a copy to avoid modifying original
   df_clean = df.copy()
   
   # Replace NaN with None
   df_clean = df_clean.replace({np.nan: None})
   
   # Remove rows that are actually footnotes (after row 100)
   df_clean = df_clean.iloc[:100]
   
   # Remove any leading/trailing spaces
   if 'NAICS  Code' in df_clean.columns:
       df_clean['NAICS  Code'] = df_clean['NAICS  Code'].str.strip() if df_clean['NAICS  Code'].dtype == 'object' else df_clean['NAICS  Code']
   
   if 'Kind of Business' in df_clean.columns:
       df_clean['Kind of Business'] = df_clean['Kind of Business'].str.strip() if df_clean['Kind of Business'].dtype == 'object' else df_clean['Kind of Business']
   
   return df_clean

def create_mrts_table(cursor):
   try:
       # Drop existing table if it exists
       cursor.execute("DROP TABLE IF EXISTS mrts_data")
       
       # Create table with adjusted column sizes
       create_table_query = """
       CREATE TABLE mrts_data (
           NAICS_Code VARCHAR(100),
           Adjustments VARCHAR(100),
           Kind_of_Business VARCHAR(500),
           Jan_2017 DECIMAL(15,2),
           Feb_2017 DECIMAL(15,2),
           Mar_2017 DECIMAL(15,2),
           Apr_2017 DECIMAL(15,2),
           May_2017 DECIMAL(15,2),
           Jun_2017 DECIMAL(15,2),
           Jul_2017 DECIMAL(15,2),
           Aug_2017 DECIMAL(15,2),
           Sep_2017 DECIMAL(15,2),
           Oct_2017 DECIMAL(15,2),
           Nov_2017 DECIMAL(15,2),
           Dec_2017 DECIMAL(15,2),
           TOTAL DECIMAL(15,2)
       )
       """
       cursor.execute(create_table_query)
       print("Table 'mrts_data' created successfully")
       
   except Exception as e:
       print(f"Error creating table: {str(e)}")

def load_data_to_mysql(cursor, cnx):
   try:
       # Read the prepared CSV file
       df = pd.read_csv(PREPARED_CSV)
       
       # Clean and prepare data
       df_clean = prepare_data_for_mysql(df)
       
       # Prepare insert query
       insert_query = """
       INSERT INTO mrts_data (
           NAICS_Code, Adjustments, Kind_of_Business, Jan_2017, Feb_2017, Mar_2017, 
           Apr_2017, May_2017, Jun_2017, Jul_2017, Aug_2017, Sep_2017, 
           Oct_2017, Nov_2017, Dec_2017, TOTAL
       ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
       """
       
       # Convert DataFrame to list of tuples for insertion
       values = df_clean.values.tolist()
       
       # Execute insert
       cursor.executemany(insert_query, values)
       
       # Commit changes
       cnx.commit()
       
       print(f"Successfully loaded {len(values)} rows into mrts_data table")
       
   except Exception as e:
       print(f"Error loading data: {str(e)}")

# Execute the installation script
print("Starting database installation process...")

# Create database and get connection
cnx, cursor = create_database()

if cnx and cursor:
   # Create table
   create_mrts_table(cursor)
   
   # Load data
   load_data_to_mysql(cursor, cnx)
   
   # Close connections
   cursor.close()
   cnx.close()
   print("Installation completed")

Starting database installation process...
Database 'retail_sales' created or already exists
Connected to retail_sales database
Table 'mrts_data' created successfully
Successfully loaded 100 rows into mrts_data table
Installation completed



[Back to top](#Index)

## 3. Analysis and Visualization

For each of the sections below, make sure you include a description of the steps you followed. Whenever possible, include screenshots of your code or program windows to demonstrate your steps.

Here, describe the differences, advantages, and disadvantages of running *queries* against your dataset using the MySQL Workbench or a Python environment.

[Back to top](#Index)

### 3.1 Running Queries in MySQL Workbech


[Back to top](#Index)

### 3.2 Running Queries From Python


[Back to top](#Index)

### 3.3 Explore Trends

Describe which *queries* you wrote the explore the differences in trends between various categories in your data.

In your submission make sure to answer the following:

- What is an economic trend and why is it considered an important measure to predict quantities, like spending patterns?
- What is the trend of the retail and food services categories? Can this data be displayed clearly or do you need to adjust some parameters to reduce extraneous details and be able to visualize a clean trend?
- When comparing businesses like bookstores, sporting goods stores, and hobbies, toys, and games stores, what is the highest trend of all of these options? Which one grew faster? Which one is higher? Is there a seasonal pattern? Were there any changes in 2020? Which is better, monthly or yearly? 

[Back to top](#Index)

### 3.4 Explore Percentage Change

Describe which *queries* you wrote to explore the differences in trends between various categories in your data.

In your submission make sure to answer the following:

- In economics, what is the percentage change and why is it considered an important measure to predict quantities like spending patterns?
- Consider the women's clothing and men's clothing businesses and their percentage change. How are these two businesses related? For each of the two businesses, what is the percentage of contribution to the whole and how does it change over time?

[Back to top](#Index)

### 3.5 Explore Rolling Time Windows


Describe which *queries* you wrote to explore the differences in trends between various categories in your data.

In your submission, make sure to answer the following:

- In economics, what is the rolling time window and why is it considered an important measure to predict quantities like spending patterns?
- Consider at least two businesses of your own from the MRTS data. Which *queries* did you write to analyze and produce graphs of rolling time windows for the chosen categories?

[Back to top](#Index)

## Conclusion

Describe your conclusions. Which one of the businesses considered seems like it's going to attract the least spending? Which business seems likely to attract the most spending? 





[Back to top](#Index
)
## References

Add all references you used to complete this project.

Use this format for articles:
- Author Last Name, Author First Name. “Article Title.” Journal Title Volume #, no. Issue # (year): page range.

- Ex: Doe, John. “Data Engineering.” Data Engineering Journal 18, no. 4 (2021): 12-18.

Use this format for websites:
- Author Last Name, Author First Name. “Title of Web Page.” Name of Website. Publishing organization, publication or revision date if available. Access date if no other date is available. URL .

- Doe, John. “Data Engineering.” Data Engineer Resource. Cengage, 2021. www.dataengineerresource.com .
